In [4]:
from os.path import dirname, join
from pprint import pprint

import pydicom
from pydicom.data import get_testdata_files
from pydicom.filereader import read_dicomdir
import pandas as pd

import numpy as np
from scipy.ndimage import zoom

from datasets import *

In [2]:
# fetch the path to the test data
filepaths = [
    '/media/mount/VIFM_DATA_1/2012/',
    '/media/mount/VIFM_DATA_1/2013/'
]
dicom_dirs = []
for path in filepaths:
    dicom_dirs.append({"dicom": read_dicomdir(path + "/DICOMDIR"), "path": path})
savepath = "./hutest/"

In [3]:
# These numbers need to be experimented with
zoom_scaling = .25
flesh_threshold = -200
bone_threshold = 220

In [4]:
i = 0

for d in dicom_dirs:
    for patient_record in d["dicom"].patient_records:
        for study in patient_record.children:
            for series in study.children:
                # Only continue if CT
                if len(series.children) > 10:
                    # Store numpies in scan list
                    flesh = []
                    bone = []
                    
                    for image in series.children:
                        # Convert to numpy
                        path = join(d["path"], *image.ReferencedFileID)
                        dcm = pydicom.read_file(path)
                        flesh_pixels = get_pixels_hu(dcm)
                        bone_pixels = get_pixels_hu(dcm)
                        
                        # Filter out air / bone
                        flesh_clean = clean_and_threshold(flesh_pixels, flesh_threshold)
                        bone_clean = clean_and_threshold(bone_pixels, bone_threshold)
                                              
                        # Append slice
                        flesh.append(flesh_clean)
                        bone.append(bone_clean)
                    
                    # Zoom, Pad to cube and save xs
                    flesh = zoom(np.asarray(flesh), zoom_scaling)
                    flesh = pad_height(flesh, flesh.shape[1])
                    np.save(savepath + "flesh/" + str(i) + ".npy", flesh)
                    
                    # Zoom, Pad to cube and save ys
                    bone = zoom(np.asarray(bone), zoom_scaling)
                    bone = pad_height(bone, bone.shape[1])
                    np.save(savepath + "bone/" + str(i) + ".npy", bone)
                    print(i)
                    i += 1

0


/home/mitchell/anaconda3/lib/python3.7/site-packages/scipy/ndimage/interpolation.py:611: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


KeyboardInterrupt: 